### Goldstein-price Function

From https://www.sfu.ca/~ssurjano/goldpr.html

The global minima is 3 in (0, -1).

Extra required packages:
- matplotlib

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from gurobi_ml.sklearn import PipelineConstr

In [ ]:
def GoldsteinPrice(x1, x2):
    fact1a = (x1 + x2 + 1) ** 2
    fact1b = 19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2
    fact1 = 1 + fact1a * fact1b

    fact2a = (2 * x1 - 3 * x2) ** 2
    fact2b = 18 - 32 * x1 + 12 * x1**2 + 48 * x2
    fact2b += -36 * x1 * x2 + 27 * x2**2
    fact2 = 30 + fact2a * fact2b
    return fact1 * fact2

In [ ]:
x = np.arange(-2, 2, 0.05)
y = np.arange(-2, 2, 0.05)
x1, x2 = np.meshgrid(x, y)
z = GoldsteinPrice(x1, x2)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(x1, x2, z, cmap=cm.coolwarm, linewidth=0.01, antialiased=False)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
X = np.concatenate([x1.ravel().reshape(-1, 1), x2.ravel().reshape(-1, 1)], axis=1)
y = z.ravel()

In [ ]:
# Run our regression
# Note that a ConvergenceWarning is not unexpected
layers = [112, 112, 112]
regression = MLPRegressor(hidden_layer_sizes=layers, activation="relu", random_state=1, max_iter=500)
pipe = make_pipeline(regression)
pipe.fit(X=X, y=y)

In [ ]:
X_test = np.random.random((100, 2)) * 4 - 2

In [ ]:
metrics.r2_score(GoldsteinPrice(X_test[:, 0], X_test[:, 1]), pipe.predict(X_test))

In [ ]:
metrics.max_error(GoldsteinPrice(X_test[:, 0], X_test[:, 1]), pipe.predict(X_test))

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(
    x1,
    x2,
    pipe.predict(X).reshape(x1.shape),
    cmap=cm.coolwarm,
    linewidth=0.01,
    antialiased=False,
)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

### Do the optimization model

In [ ]:
npoints = 1
optfeat = [0, 1]

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((npoints, len(optfeat)), lb=-2, ub=2, name="x")
y = m.addMVar((npoints, 1), lb=-GRB.INFINITY, name="y")

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

# Add constraint to predict value of y using x
PipelineConstr(m, pipe, x, y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 30
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

### Look at the solution

In [ ]:
x.X

In [ ]:
GoldsteinPrice(x.X[0, 0], x.X[0, 1])

In [ ]:
y.X

### Optimize it as a regular Gurobi Model
We can also create this model without involving sklearn. Note that this will take significantly longer

In [ ]:
m2 = gp.Model()
x1 = m2.addVar(lb=-2, ub=2)
x2 = m2.addVar(lb=-2, ub=2)
fact1a = m2.addVar(lb=-GRB.INFINITY)
fact1b = m2.addVar(lb=-GRB.INFINITY)
fact1 = m2.addVar(lb=-GRB.INFINITY)
fact2a = m2.addVar(lb=-GRB.INFINITY)
fact2b = m2.addVar(lb=-GRB.INFINITY)
fact2 = m2.addVar(lb=-GRB.INFINITY)

In [ ]:
m2.setObjective(fact1 * fact2)

m2.addConstr(fact1a == (x1 + x2 + 1) ** 2)

m2.addConstr(fact1b == 19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2)
m2.addConstr(fact1 == 1 + fact1a * fact1b)

m2.addConstr(fact2a == (2 * x1 - 3 * x2) ** 2)
m2.addConstr(fact2b == 18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2)
m2.addConstr(fact2 == 30 + fact2a * fact2b)

In [ ]:
m2.Params.NonConvex = 2
m2.Params.TimeLimit = 30
m2.optimize()

In [ ]:
x1.X

In [ ]:
x2.X

Copyright © 2022 Gurobi Optimization, LLC